In [1]:
import requests_cache

import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import date, timedelta
import os
import csv
from datetime import datetime
from dateutil import parser

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

In [2]:
requests_cache.install_cache('demo_cache',expire_after=None,allowable_methods=['GET'])
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been 

In [3]:
link = "https://www.worldometers.info/coronavirus/"

driver.get(link)

def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

In [4]:
header_by_day = ["#","Country","Date","TotalCases","NewCases","TotalDeaths","NewDeaths","TotalRecovered","NewRecovered",
"ActiveCases","Serious,Critical","Tot Cases/1M pop","Deaths/1M pop","TotalTests","Tests/1M pop","Population","Continent"]

header_by_week = ["#", "Country", "Cases in the last 7 days", "Cases in the preceding 7 days",
"Weekly case /%/ change", "Cases in the last 7 days/1M pop", "Deaths in the last 7 days", 
"Deaths in the preceding 7 days", "Weekly Death /%/ change", "Deaths in the last 7 days/1M pop", "Population", "Continent"]

In [5]:
row_total = driver.find_elements(By.XPATH,"//tr[@class='even'] | //tr[@class='odd']")

rows_past = []
rows_past_modify = []
rows_yesterday = []
rows_two_days_ago = []
count = 0
row_count = 226

for n in row_total:
    n = n.get_attribute('textContent')
    if count == row_count:
        rows_past.append(n.split("\n"))
    else:
        count+=1

for n in rows_past:
    n.pop(0)
    n = n[: len(n) - 6]
    rows_past_modify.append(n)

rows_yesterday,rows_two_days_ago = split_list(rows_past_modify)

rows_yesterday = [ele for ele in rows_yesterday if ele != []]
rows_two_days_ago = [ele for ele in rows_two_days_ago if ele != []]

In [6]:
today = date.today()
yesterday = (today - timedelta(1)).strftime("%d-%m-%Y")
two_days_ago = (today - timedelta(2)).strftime("%d-%m-%Y")

for row in rows_yesterday:

    row.insert(2,yesterday)
for row in rows_two_days_ago:

    row.insert(2,two_days_ago)

total_covid = rows_two_days_ago + rows_yesterday 

Check xem có dòng nào bị dư giá trị rỗng không, nếu có cần chạy lại từ đầu

In [7]:
for i in total_covid:
    if len(i) > 17:
        if i[-1] == '' or i[-1] == ' ':
            i.pop(-1)
        else:
            print(i)

In [8]:
df_covid = pd.DataFrame(data=rows_yesterday,columns=header_by_day)
df_covid

,#,Country,Date,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent
0,1,USA,03-03-2022,"80,794,479","+52,593","980,471","+1,885","53,945,789","+206,050","25,868,219","7,161","241,728","2,933","952,798,272","2,850,671","334,236,503",North America
1,2,India,03-03-2022,"42,945,160","+6,561","514,419",+173,"42,353,620","+14,947","77,121","8,944","30,618",367,"769,167,052","548,384","1,402,606,131",Asia
2,3,Brazil,03-03-2022,"28,842,160","+30,995","650,052",+335,"26,668,010","+162,005","1,524,098","8,318","134,102","3,022","63,776,166","296,528","215,076,719",South America
3,4,France,03-03-2022,"22,840,306","+57,697","138,762",+186,"21,233,702","+152,816","1,467,842","2,484","348,632","2,118","246,629,975","3,764,529","65,514,171",Europe
4,5,UK,03-03-2022,"19,029,040","+43,923","161,704",+74,"17,537,214",,"1,330,122",278,"277,877","2,361","483,496,650","7,060,391","68,480,157",Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,222,Western Sahara,03-03-2022,10,,1,,8,,1,,16,2,,,"621,783",Africa
222,223,MS Zaandam,03-03-2022,9,,2,,7,,0,,,,,,,
223,224,Marshall Islands,03-03-2022,7,,,,7,,0,,117,,,,"59,855",Australia/Oceania
224,225,Saint Helena,03-03-2022,2,,,,2,,0,,327,,,,"6,107",Africa


In [9]:
df_covid = pd.DataFrame(data=rows_yesterday,columns=header_by_day)
if not os.path.isfile('Corona_by_day.tsv'):
    df_covid = pd.DataFrame(data=total_covid,columns=header_by_day)
    df_covid.to_csv("Corona_by_day.tsv", sep = "\t", mode = "a", index = False)
else:
    df_covid.to_csv("Corona_by_day.tsv", sep = "\t", mode = "a", index = False,header=False)

In [10]:
df_covid_by_day = pd.read_csv('Corona_by_day.tsv',sep='\t')

In [11]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def convert_numeric(num):
    condition = {",":"","+":"","-":"","%":""}
    if num == "nan" or num == " " or num == "  " or num == " N/A":
        return 0

    for i, j in condition.items():
        num = num.replace(i, j)
    if RepresentsInt(num) == True:
        return int(num)
    return float(num)

In [12]:
for i in range(3,16):
    df_covid_by_day[header_by_day[i]] = df_covid_by_day[header_by_day[i]].apply(lambda x:convert_numeric(str(x)))
    
df_covid_by_day[header_by_day[16]] = df_covid_by_day[header_by_day[16]].apply(lambda x:str(x).replace(" ",""))
df_covid_by_day['Date'] = pd.to_datetime(df_covid_by_day['Date'],format="%d-%m-%Y")

In [13]:
df_covid_by_day

,#,Country,Date,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent
0,1,USA,2022-02-28,80656025,34828,975951,843,53500262,278579,26179812,7770,241322,2920,950614834,2844228,334225917,NorthAmerica
1,2,India,2022-02-28,42931045,6915,514054,211,42324550,16864,92441,8944,30610,367,767481346,547211,1402532051,Asia
2,3,Brazil,2022-02-28,28787620,19516,649443,248,26336373,152750,1801804,8318,133853,3020,63776166,296539,215068475,SouthAmerica
3,4,France,2022-02-28,22702815,13483,138367,232,20901504,235191,1662944,2484,346537,2112,246629975,3764574,65513384,Europe
4,5,UK,2022-02-28,18886701,27312,161361,46,17449650,98709,1275690,289,275806,2356,480341745,7014519,68478212,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,222,Western Sahara,2022-03-03,10,0,1,0,8,0,1,0,16,2,0,0,621783,Africa
900,223,MS Zaandam,2022-03-03,9,0,2,0,7,0,0,0,0,0,0,0,0,nan
901,224,Marshall Islands,2022-03-03,7,0,0,0,7,0,0,0,117,0,0,0,59855,Australia/Oceania
902,225,Saint Helena,2022-03-03,2,0,0,0,2,0,0,0,327,0,0,0,6107,Africa


In [14]:
df_covid_by_day.dtypes

#                            int64
Country                     object
Date                datetime64[ns]
TotalCases                   int64
NewCases                     int64
TotalDeaths                  int64
NewDeaths                    int64
TotalRecovered               int64
NewRecovered                 int64
ActiveCases                  int64
Serious,Critical             int64
Tot Cases/1M pop             int64
Deaths/1M pop                int64
TotalTests                   int64
Tests/1M pop                 int64
Population                   int64
Continent                   object
dtype: object

Crawl weekly data

In [15]:
link = "https://www.worldometers.info/coronavirus/weekly-trends/#weekly_table"
driver.get(link)
weekly_data = driver.find_elements(By.XPATH,"//tr[@class='even'] | //tr[@class='odd']")

In [16]:
for n in weekly_data:
    n = n.get_attribute('textContent').split("\n")
    n[:] = [x.strip() for x in n if x.strip()]
    if len(n) != len(header_by_week):
        print("Error Header in",n)

In [17]:
row_week_trend = []

for n in weekly_data:
    n = n.get_attribute('textContent').split("\n")
    n[:] = [x.strip() for x in n if x.strip()]
    row_week_trend.append(n)
    if len(n) != len(header_by_week):
        print("Error Header in",n)

In [18]:
df_covid_by_week = pd.DataFrame(data=row_week_trend,columns=header_by_week)
df_covid_by_week.to_csv("Corona_by_week.tsv", sep = "\t", mode = "w", index = False)

In [19]:
df_covid_by_week

,#,Country,Cases in the last 7 days,Cases in the preceding 7 days,Weekly case /%/ change,Cases in the last 7 days/1M pop,Deaths in the last 7 days,Deaths in the preceding 7 days,Weekly Death /%/ change,Deaths in the last 7 days/1M pop,Population,Continent
0,1,S. Korea,"1,163,504","776,331",+50%,"22,662",659,405,+63%,13,"51,342,704",Asia
1,2,Germany,"1,080,549","1,165,693",-7%,"12,829","1,354","1,419",-5%,16,"84,230,316",Europe
2,3,Russia,"797,254","1,135,690",-30%,"5,459","5,414","5,433",0%,37,"146,038,728",Europe
3,4,Vietnam,"638,613","350,850",+82%,"6,464",679,585,+16%,7,"98,797,607",Asia
4,5,Japan,"460,706","551,354",-16%,"3,661","1,588","1,571",+1%,13,"125,831,319",Asia
...,...,...,...,...,...,...,...,...,...,...,...,...
211,212,Cayman Islands,0,"1,686",-100%,0,0,1,-100%,0,"67,008",North America
212,213,Falkland Islands,0,4,-100%,0,0,0,0%,0,"3,649",South America
213,214,Guadeloupe,0,"7,108",-100%,0,0,29,-100%,0,"400,238",North America
214,215,Tajikistan,0,3,-100%,0,0,0,0%,0,"9,894,249",Asia


In [20]:
header_by_week[2]

'Cases in the last 7 days'

In [21]:
for i in range(2,11):
    df_covid_by_week[header_by_week[i]] = df_covid_by_week[header_by_week[i]].apply(lambda x:convert_numeric(str(x)))
    
df_covid_by_week[header_by_week[-1]] = df_covid_by_week[header_by_week[-1]].apply(lambda x:str(x).replace(" ",""))

In [22]:
df_covid_by_week

,#,Country,Cases in the last 7 days,Cases in the preceding 7 days,Weekly case /%/ change,Cases in the last 7 days/1M pop,Deaths in the last 7 days,Deaths in the preceding 7 days,Weekly Death /%/ change,Deaths in the last 7 days/1M pop,Population,Continent
0,1,S. Korea,1163504,776331,50.0,22662.0,659,405,63,13.0,51342704,Asia
1,2,Germany,1080549,1165693,7.0,12829.0,1354,1419,5,16.0,84230316,Europe
2,3,Russia,797254,1135690,30.0,5459.0,5414,5433,0,37.0,146038728,Europe
3,4,Vietnam,638613,350850,82.0,6464.0,679,585,16,7.0,98797607,Asia
4,5,Japan,460706,551354,16.0,3661.0,1588,1571,1,13.0,125831319,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...
211,212,Cayman Islands,0,1686,100.0,0.0,0,1,100,0.0,67008,NorthAmerica
212,213,Falkland Islands,0,4,100.0,0.0,0,0,0,0.0,3649,SouthAmerica
213,214,Guadeloupe,0,7108,100.0,0.0,0,29,100,0.0,400238,NorthAmerica
214,215,Tajikistan,0,3,100.0,0.0,0,0,0,0.0,9894249,Asia


In [23]:
df_covid_by_week.dtypes

#                                    object
Country                              object
Cases in the last 7 days              int64
Cases in the preceding 7 days         int64
Weekly case /%/ change              float64
Cases in the last 7 days/1M pop     float64
Deaths in the last 7 days             int64
Deaths in the preceding 7 days        int64
Weekly Death /%/ change               int64
Deaths in the last 7 days/1M pop    float64
Population                            int64
Continent                            object
dtype: object

Đã xong:
- Crawl Data day và week
- Tiền xử lý các trường thành dữ liệu số

Đợi ngày mai test lần nữa để chắc chắn.
Đã test hai ngày.
Hay gặp phải lỗi bị dư khoảng trắng '' khi crawl, đang tìm cách fix, bây giờ nếu bị lỗi phải chạy lại chừng nào lấy đc thì thôi